In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn')
plt.rcParams.update({'font.size': 18})
sns.set(font_scale=2.5)

import missingno as msno
from sklearn.preprocessing import LabelEncoder

#ignore warnings 
import warnings
warnings.filterwarnings('ignore')

%matplotlib inlinethe current session

In [ ]:
df_train = pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
df_train.head()

In [ ]:
df_test = pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')
df_test.head()

In [ ]:
for col in df_train.columns : 
    msg = 'columnn : {:>10}\t count of NaN value : {:.0f}'.format(col, 100 * (df_train[col].isnull().sum() ))
    print(msg)

In [ ]:
for col in df_test.columns : 
    msg = 'columnn : {:>10}\t count of NaN value : {:.0f}'.format(col, 100 * (df_test[col].isnull().sum() ))
    print(msg)

In [ ]:
df_train.info()

In [ ]:
df_train.shape

In [ ]:
df_train = df_train.drop(['id'], axis=1)
df_test = df_test.drop(['id'], axis=1)

In [ ]:
int_column = df_train.dtypes[df_train.dtypes =='int64'].index |  df_train.dtypes[df_train.dtypes =='float64'].index

In [ ]:
for col in int_column : 
    plt.figure(figsize=(12,4))
    
    plt.subplot(1,2,1)
    sns.distplot(df_train[col])
    plt.xlabel(col)
    plt.ylabel('Density')
    
    plt.subplot(1,2,2)
    sns.boxplot(x='Response', y = col, data =df_train, showmeans = True)
    plt.xlabel('Target')
    plt.ylabel(col)
    
    plt.show()

In [ ]:
obj_column = df_train.dtypes[df_train.dtypes == 'object'].index
for i in range(0, len(obj_column)) :
    print(obj_column[i])
    print(df_train[obj_column[i]].unique())
    print()

In [ ]:
for i in range(0, len(obj_column)) :
    fig, ax = plt.subplots(figsize=(15,4))

    sns.countplot(x = obj_column[i], data = df_train)
    sns.set(font_scale=1)

    ax.set_title('{} Count Distribution'.format(obj_column[i]))

In [ ]:
df_train.loc[df_train['Gender'] == 'Male', 'Gender'] = 0
df_train.loc[df_train['Gender'] == 'Female', 'Gender'] = 1
df_test.loc[df_test['Gender'] == 'Male', 'Gender'] = 0
df_test.loc[df_test['Gender'] == 'Female', 'Gender'] = 1

df_train.loc[df_train['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = 2
df_train.loc[df_train['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = 1
df_train.loc[df_train['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = 0
df_test.loc[df_test['Vehicle_Age'] == '> 2 Years', 'Vehicle_Age'] = 2
df_test.loc[df_test['Vehicle_Age'] == '1-2 Year', 'Vehicle_Age'] = 1
df_test.loc[df_test['Vehicle_Age'] == '< 1 Year', 'Vehicle_Age'] = 0

df_train.loc[df_train['Vehicle_Damage'] == 'Yes', 'Vehicle_Damage'] = 1
df_train.loc[df_train['Vehicle_Damage'] == 'No', 'Vehicle_Damage'] = 0
df_test.loc[df_test['Vehicle_Damage'] == 'Yes', 'Vehicle_Damage'] = 1
df_test.loc[df_test['Vehicle_Damage'] == 'No', 'Vehicle_Damage'] = 0

In [ ]:
pd.set_option('max_columns', None)
df_train.head()

In [ ]:
pd.set_option('max_columns', None)
df_test.head()

In [ ]:
heatmap_data = df_train

colormap = plt.cm.RdBu
plt.figure(figsize=(20,20))
plt.title('Pearson Correlation of Features', y = 1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, vmax=1.0, square=True, 
            cmap=colormap, linecolor='white', annot=True, annot_kws={'size':16})

del heatmap_data


In [ ]:
df_train.info()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier

In [ ]:
df_train

In [ ]:
df_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_train.drop('Response',axis=1),
                                                    df_train['Response'],
                                                    test_size=.3, random_state = 42,
                                                    stratify= df_train['Response'])

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train = pd.DataFrame(X_train, columns=df_train.drop('Response',axis=1).columns)
X_test = pd.DataFrame(X_test, columns=df_train.drop('Response',axis=1).columns)

In [ ]:
X_train

In [ ]:
from sklearn.linear_model import LogisticRegression
models = [LogisticRegression(),
          DecisionTreeClassifier(),
          RandomForestClassifier(),
          XGBClassifier()]

names = [ 'LogisticRegression',
         'DecisionTreeClassifier',
          'RandomForestClassifier',
          'XGBClassifier']

for model,name in zip(models,names):
    m = model.fit(X_train,y_train)
    print(name, 'report:')
    print('Train score',model.score(X_train,y_train))
    print('Test score',model.score(X_test,y_test))
    print()
    print("Train confusion matrix:\n",confusion_matrix(y_train, model.predict(X_train)),'\n')
    print("Test confusion matrix:\n",confusion_matrix(y_test, model.predict(X_test)))
    print('*'*50)

In [ ]:
model = DecisionTreeClassifier(max_depth=3)
model.fit(X_train, y_train)

from sklearn.tree import plot_tree
plt.figure(figsize=(20,15))
plot_tree(model,
          feature_names= df_train.drop('Response', axis=1).columns,  
          class_names= ['yes','no'],
          filled=True)
plt.show()

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import accuracy_score

m = LogisticRegression().fit(X_train,y_train)
pred_y = m.predict(X_test)
print('*'*50)
print('Report')
print('model : LogisticRegression')
print('Train score',m.score(X_train,y_train))
print('Test score',m.score(X_test,y_test))
print()
print("accuracy: %.2f" %accuracy_score(y_test, pred_y))
print("Precision : %.3f" % precision_score(y_test, pred_y))
print("Recall : %.3f" % recall_score(y_test, pred_y))
print("F1 : %.3f" % f1_score(y_test, pred_y))
print()
print("Train confusion matrix:\n",confusion_matrix(y_train, m.predict(X_train)),'\n')
print("Test confusion matrix:\n",confusion_matrix(y_test, m.predict(X_test)))
print('*'*50)

In [ ]:
lr_clf = LogisticRegression(solver='liblinear')
lr_clf.fit(X_train, y_train)

print('Train score',lr_clf.score(X_train,y_train))
print('Test score',lr_clf.score(X_test,y_test))

lrCoef = LogisticRegression().fit(X_train,y_train).coef_
print(lrCoef)

In [ ]:
print("Coefficient of Logistic Regression")
for i in range(0, len(lrCoef[0])) :
    print('{} : {}'.format(X_train.columns[i], lrCoef[0][i]))

In [ ]:
coefdf = pd.DataFrame(data=X_train.columns, index=range(0, len(lrCoef[0])), columns=['Feature'])
coefdf['Coef'] = lrCoef[0]
coefdf['Absuolute num of Coef'] = abs(lrCoef[0])
coefdf = coefdf.sort_values(by='Absuolute num of Coef', ascending=False).reset_index(drop=True)
coefdf

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
sns.barplot(data=coefdf, y=coefdf['Feature'], x=coefdf['Coef'])

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.title('Coefficient of Logistic Regression\n(score : 88%)', fontsize=20)
plt.xlabel('Coefficient')

plt.savefig('Coefficient of Logistic Regression.png')
plt.show()

In [ ]:
import shap

In [ ]:
XGBmodel = XGBClassifier().fit(X_train, y_train)

explainer = shap.TreeExplainer(XGBmodel)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test, plot_size=(10,5), show=False)
plt.title('SHAP of XGBoost Classifier\n(score : 88%)')
plt.figsize=(10,5)
plt.show()

In [ ]:
RFmodel = RandomForestClassifier().fit(X_train, y_train)

In [ ]:
from sklearn.inspection import permutation_importance

result = permutation_importance(RFmodel, X_test, y_test, n_repeats=10, random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()

plt.figure(figsize=(10,5))
plt.title('Permutation Importance of Random Forest Classifier\n(score : 87%)')

plt.xticks(fontsize=12)
plt.yticks(fontsize=12)

plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=X_test.columns[sorted_idx]);

In [ ]:
# RFmodel = RandomForestClassifier().fit(X_train, y_train)

# explainer = shap.TreeExplainer(RFmodel)
# shap_values = explainer.shap_values(X_test)

# shap.summary_plot(shap_values, X_test, plot_size=(10,5), show=False)
# plt.title('SHAP of Random Forest Classifier\n(score : 87%)')
# plt.show()

Whether already **has insurance or not**(Previously_Insured) was the most important attribute to predict the customer's response.

Whether someone's vehicle got damaged or not was also significant feature as much as 'Previously_Insured'.

The older the vehicle is, the more likely to be interested.

Older people tend to pay more attention about insurance than young people do.

How long the customer has been associated with the company('Vintage') or how did reach to the customer('Policy_Sales_Channel'), even gender or region were not that influential.

In conclusion, targeting on the older customer how got damaged old vehichle would be the most cost-efficient marketing strategy no matter how(e-mail, phone call, in person or wathever it is).